In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from datetime import datetime
import warnings
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import json

from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
tf.random.set_seed(13)

In [ ]:
datetime.now().strftime("%m-%d-%Y_%Hh%Mmin%Ss")

In [ ]:
global_epoch_number = 30

## Reading data

In [ ]:
# reading data
evo_data = pd.read_csv('data/interpol/evo_interpol_demand.csv', index_col=0)
modo_data = pd.read_csv('data/interpol/modo_interpol_demand.csv', index_col=0)
c2g_data = pd.read_csv('data/interpol/c2g_interpol_demand.csv', index_col=0)

In [ ]:
evo_data.columns

In [ ]:
evo_data.drop(columns = ['hour_0', 'hour_1', 'hour_2', 'hour_3',
       'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22',
       'hour_23', "interpolate"], inplace=True)
modo_data.drop(columns = ['hour_0', 'hour_1', 'hour_2', 'hour_3',
       'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22',
       'hour_23', "interpolate"], inplace=True)
c2g_data.drop(columns = ['hour_0', 'hour_1', 'hour_2', 'hour_3',
       'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22',
       'hour_23', "interpolate"], inplace=True)

In [ ]:
unievo_data = pd.DataFrame(evo_data.travels)
unimodo_data = pd.DataFrame(modo_data.travels)
unic2g_data = pd.DataFrame(c2g_data.travels)

In [ ]:
# Adding the Canada Day Holiday
evo_data.index = pd.to_datetime(evo_data.index)
modo_data.index = pd.to_datetime(modo_data.index)
c2g_data.index = pd.to_datetime(c2g_data.index)

evo_data["holidays"] = pd.Series()
modo_data["holidays"] = pd.Series()
c2g_data["holidays"] = pd.Series()

evo_data["holidays"] = evo_data["holidays"].fillna(0)
modo_data["holidays"] = modo_data["holidays"].fillna(0)
c2g_data["holidays"] = c2g_data["holidays"].fillna(0)

canada_day = datetime(2018, 7, 1)
end_canada_day = datetime(2018,7 ,3)

evo_data.loc[((evo_data.index > canada_day) & (evo_data.index <= end_canada_day))]["holidays"] = 1
modo_data.loc[((modo_data.index > canada_day) & (modo_data.index <= end_canada_day))]["holidays"] = 1
c2g_data.loc[((c2g_data.index > canada_day) & (c2g_data.index <= end_canada_day))]["holidays"] = 1

In [ ]:
init_period = '06-23'
end_period = '07-12'

evo_data = evo_data[(evo_data.index >= '2018-'+init_period) & (evo_data.index <= '2018-'+end_period)]
modo_data = modo_data[(modo_data.index >= '2018-'+init_period) & (modo_data.index <= '2018-'+end_period)]
c2g_data = c2g_data.loc["2016-12-13 15:00:00":"2017-02-25 17:00:00"]

unievo_data = unievo_data[(unievo_data.index >= '2018-'+init_period) & (unievo_data.index <= '2018-'+end_period)]
unimodo_data = unimodo_data[(unimodo_data.index >= '2018-'+init_period) & (unimodo_data.index <= '2018-'+end_period)]
unic2g_data = unic2g_data.loc["2016-12-13 15:00:00":"2017-02-25 17:00:00"]

## LSTM Data Preparation

In [ ]:
def sup_learning_formatter(data, past_lags, future_steps, future_steps_skipped, train_split, all_parameters_predicted):
    X = []
    y = []
    
    norm_data = data.values
    travels_data = data.travels.values
    
    if(all_parameters_predicted):
        for n in range(len(data) - past_lags - future_steps):
            X.append(norm_data[n : n + past_lags])
            y.append(norm_data[n + past_lags : n + past_lags + 1])
        return np.array(X), np.squeeze(np.array(y))
    
    else:
        for n in range(len(data) - past_lags - future_steps - future_steps_skipped):
            X.append(norm_data[n : n + past_lags])
            y.append(travels_data[n + past_lags  + future_steps_skipped: n + past_lags + future_steps + future_steps_skipped])
            
        return np.array(X), np.array(y)

In [ ]:
def train_val_test_splitter(data, splits):
    locs = [int(len(data)*n) for n in splits]
    return data[:locs[0]], data[locs[0]:locs[1]], data[locs[1]:], data[0].shape

In [ ]:
def eval_model(y, y_hat):
    evaluation = {}
    evaluation["RMSE"] = np.sqrt(mean_squared_error(y, y_hat))
    evaluation["MAE"] = mean_absolute_error(y, y_hat)
    evaluation["R2"] = r2_score(y, y_hat)

    return evaluation

In [ ]:
def persistance_model(X, timesteps):
    y_hat = []
    for x in X:
        y_hat.append(np.array([x[-1][0] for _ in range(timesteps)]))

    return np.array(y_hat)

# Training Models

In [ ]:
def plot_train_history(history, title, save_file=False):
    history = pd.DataFrame(history.history)

    history.plot(figsize=(8, 5))
    plt.grid(True)
    plt.savefig("plots\\" + title.replace(" ", "_") + ".png", bbox_inches='tight') if save_file else print()
    plt.show()

Next, will be generated the model for each dataset

## Grid Search

In [ ]:
class GridSearchLSTM:
    def __init__(self):
        self.evaluations = pd.DataFrame()
        self.best_estimator = None

    def search(self, feature_dict, data, verbose=1, windows=1, splits = (0.6, 0.8), past_lags=24, future_steps=12, future_steps_skipped=0):
    # Essa função faz igual a do sklearn, eu só adaptei pro lstm
    # Eu não tenho certeza se pra mais de uma window tá funcionando perfeitamente
        
        def average_evaluations(validation_eval, key="val_loss"):
            # Essa função é usada para ordenar a lista
            acc_value = 0
            for split, evaluation in validation_eval:
                acc_value += evaluation[key]
            return acc_value/len(validation_eval)
        
        # Aqui em baixo ele só gera os dicionarios com os casos de teste e em seguida percorre todos testando e salvando a performance
        possibilities_list = self._create_feature_dict(feature_dict)
        current_evaluations = []
        if(windows == 1):
            for test in tqdm(possibilities_list):
                model, hist, test_data, evaluation = self.run_lstm(data, past_lags, future_steps, future_steps_skipped, splits, verbose=verbose, **test)
                validation_eval = {key:value[-1] for key, value in hist.history.items()}
                current_evaluations.append([test, validation_eval])
        
        else:
            increase = splits[1]/(windows + 1)
            for test in tqdm(possibilities_list):
                validation_eval = []
                for i in tqdm(range(windows)):
                    cur_split = (increase*(i + 1), increase*(i + 2))
                    model, hist, test_data, evaluation = self.run_lstm(data, past_lags, future_steps, future_steps_skipped, cur_split, verbose=verbose, **test)
                    cur_validation_eval = {key:value[-1] for key, value in hist.history.items()}
                    validation_eval.append([cur_split, cur_validation_eval])
                    print(cur_split)
                current_evaluations.append([test, validation_eval])
        
        if(windows == 1):
            current_evaluations.sort(key=lambda x: x[1]["val_loss"])
        else:
            current_evaluations.sort(key=lambda x: average_evaluations(x[1]))   
        
        print(current_evaluations[0][1])
        self.evaluations = pd.DataFrame(map(lambda x: {**x[0], **x[1]}, current_evaluations))
        self.best_estimator = current_evaluations[0][0]
            

    def _create_feature_dict(self, feature_dict):
        return self._create_feature_dict_recurse({}, feature_dict, list(feature_dict.keys()))

    def _create_feature_dict_recurse(self, start_dict, feature_dict, remaining_keys):
        if len(remaining_keys) == 0:
            return [start_dict]
        new_feature_dict = feature_dict.copy()
        returned_list = []
        del new_feature_dict[remaining_keys[0]]
        for item in feature_dict[remaining_keys[0]]:
            new_start_dict = start_dict.copy()
            new_start_dict[remaining_keys[0]] = item
            returned_list += self._create_feature_dict_recurse(new_start_dict, new_feature_dict, remaining_keys[1:])
        return returned_list


    def run_lstm(self, data, past_lags, future_steps, future_steps_skipped, splits, all_parameters_predicted=False, node_number=50,
                 epochs=10, batch_size=64, loss='mae', dropout=0.5, layer_count=2, verbose=1):
        
        X, y = sup_learning_formatter(data, past_lags, future_steps, future_steps_skipped, splits[0], all_parameters_predicted)
        X_train, X_val, X_test, X_shape = train_val_test_splitter(X, splits)
        y_train, y_val, y_test, y_shape = train_val_test_splitter(y, splits)


        train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        train = train.cache().shuffle(batch_size).batch(batch_size).repeat()

        val = tf.data.Dataset.from_tensor_slices((X_val, y_val))
        val = val.batch(batch_size).repeat()

        model = tf.keras.models.Sequential()

        if(layer_count == 1):
            model.add(tf.keras.layers.LSTM(node_number,
                                    input_shape=X_shape))
            model.add(tf.keras.layers.Dropout(dropout))
        else:
            model.add(tf.keras.layers.LSTM(node_number, return_sequences=True,
                                    input_shape=X_shape))
            model.add(tf.keras.layers.Dropout(dropout))

            for _ in range(layer_count - 2):
                model.add(tf.keras.layers.LSTM(node_number, return_sequences=True, activation='relu'))

            model.add(tf.keras.layers.LSTM(node_number, activation='relu'))
        
        model.add(tf.keras.layers.Dense(y_shape[0]))
        
        def rmse(y_true, y_pred):
            return tf.sqrt(tf.reduce_mean((y_true - y_pred)**2))

        model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss=loss, metrics=[rmse])
        
        
        print(X.shape, y.shape)
        history = model.fit(train, epochs=epochs, steps_per_epoch=50,
                            validation_data=val, validation_steps=50, verbose=verbose
                            )
        y_hat_test = model.predict(X_test)
        evaluation = eval_model(y_test, y_hat_test)

        return model, history, (X_test, y_test), evaluation


In [ ]:
grid_search = GridSearchLSTM()

## Test Windows

In [ ]:
feature_dict = {"epochs":[25],
                "layer_count":[2, 3], 
                "node_number":[140, 160, 180], 
                "dropout":[0.3, 0.5, 0.7], 
                "all_parameters_predicted":[False]
}

grid_search.search(feature_dict, unic2g_data, windows=1, future_steps=1)

In [ ]:
def evaluator(x):
    return sum([splits[-1]*values['val_loss'] for (splits, values) in x])

In [ ]:
grid_search.evaluations

In [ ]:
grid_search.evaluations.to_csv(f'results\\GridSearch_Results\\future_12hrs_unic2g_grid_search_{datetime.now().strftime("%m-%d-%Y_%Hh%Mmin%Ss")}.csv')